# Pathfinder for Hidden connections 

#https://neo4j.com/docs/cypher-manual/current/introduction/uniqueness/ 
#https://stackoverflow.com/questions/14345555/return-unique-nodes-in-cypher-path-query
#https://github.com/neo4j/neo4j/issues/12628
#How to improve it to just get once Episode I: The Phantom Menace?

In [18]:
#Imports
from neo4j import GraphDatabase, basic_auth
import neo4jupyter
import pandas as pd
import sys  
sys.path.insert(0, '')
from vis_class import vis_class

In [19]:
#Create db connector
login = open("Login.txt", "r")
uri = login.readline()
user = login.readline()
password = ""
login.close()

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")

## General Path finding / matching


Here we get an short overview how characters are connected over max. 5 diffrent relations

In [11]:
five_path = '''MATCH p = ()-[*5]-() RETURN nodes(p) Limit 5'''
results = session.read_transaction(
    lambda tx: tx.run(five_path).data())
results

[{'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'BRAVO THREE'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'GREEDO'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'OBI-WAN'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': 'EMPEROR'}]},
 {'nodes(p)': [{'name': 'Episode I: The Phantom Menace'},
   {'name': 'JAR JAR'},
   {'name': 'JABBA'},
   {'name': 'FODE/BEED'},
   {'name': 'Episode I: The Phantom Menace'},
   {'name': '

## Finding hidden path

Something like that should it be
match (m:Movie) 
where m.name="Episode VI: Return of the Jedi" 
match (p:Person) 
where p.name="KI-ADI-MUNDI" 
match  
searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
return distinct m.name,searchPath 


'''match (m:Movie) 
                    where m.name="Episode II: Attack of the Clones" 
                  match (p:Person) 
                    where p.name="KI-ADI-MUNDI" 
                  match searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
                    return searchPath'''

In [15]:
person_movie = '''match searchPath=(ms:Movie)<-[*]-()-[*]->(ps:Person) 
                    WHERE ms.name="Episode II: Attack of the Clones" AND ps.name="KI-ADI-MUNDI" 
                    return DISTINCT searchPath'''

results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

Transaction failed and will be retried in 1.065840189372898s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 2.295833319399573s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 3.908036394459752s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 8.229954627003293s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 17.1075383364218s (The allocation of an extra 2.0 MiB would us

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached}

In [16]:
person_movie = '''match searchPath=(ms:Movie)<-[*]->(ps:Person) 
                    WHERE ms.name="Episode II: Attack of the Clones" AND ps.name="KI-ADI-MUNDI" 
                    return DISTINCT searchPath'''

results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

Failed to read from defunct connection IPv4Address(('46753aad.databases.neo4j.io', 7687)) (IPv4Address(('34.78.76.49', 7687)))
Transaction failed and will be retried in 0.8505094121104508s (Failed to read from defunct connection IPv4Address(('46753aad.databases.neo4j.io', 7687)) (IPv4Address(('34.78.76.49', 7687))))


ValueError: Cannot resolve address 46753aad.databases.neo4j.io:7687

In [14]:
#Find path between Person and Movie
person_movie = '''match (m:Movie) 
                    where m.name="Episode II: Attack of the Clones" 
                  match (p:Person) 
                    where p.name="KI-ADI-MUNDI" 
                  match searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
                    return searchPath'''
results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())


Transaction failed and will be retried in 0.8575031020814184s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 1.8814436571573532s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 3.8677699687684233s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 8.168592139121444s (The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 13.36056847305675s (The allocation of an extra 2.0 MiB woul

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 2.0 MiB would use more than the limit 250.0 MiB. Currently using 250.0 MiB. dbms.memory.transaction.global_max_size threshold reached}

In [ ]:
#Find path between Person and Person which are not related at the first glance

In [ ]:
#Find path between Movie and Movie which are not related at the first glance

## Closing the connection to the database

In [20]:
#Close all connections
driver.close()
session.close()
drive = None
session = None